In [ ]:
# Regular Package
# https://www.kaggle.com/code/moathmohamed/twitter-sentiment-lstm-98

import io
import csv
import re
import string
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from collections import Counter
from string import *
import seaborn as sn

# nltk
import nltk
from nltk.stem.snowball import SnowballStemmer
from nltk.probability import FreqDist
from nltk.tokenize import word_tokenize
from nltk.corpus import *
from nltk.stem import PorterStemmer
from nltk.stem import WordNetLemmatizer
from nltk.tokenize import *
from gensim import *

nltk.download('stopwords')
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')
nltk.download('wordnet')
nltk.download('omw-1.4')

# sklearn
from sklearn.feature_extraction.text import *
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.utils import shuffle

# WordCloud
from PIL import Image
from wordcloud import WordCloud, STOPWORDS, ImageColorGenerator
from collections import Counter, defaultdict

# Word2vec
import gensim
from gensim.test.utils import common_texts
from gensim.models import Word2Vec

#matplolib
import matplotlib.pyplot as plt

# Tensorflow and keras
!pip install tensorflow
import tensorflow as tf
from tensorflow import keras
#from tensorflow.keras import layers

!pip install keras
from keras_preprocessing.text import Tokenizer
from keras_preprocessing.sequence import pad_sequences
#from keras.models import Sequential
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Activation, Dense, Dropout, Embedding, Flatten, Conv1D, MaxPooling1D, LSTM, Bidirectional
#from keras.layers import Activation, Dense, Dropout, Embedding, Flatten, Conv1D, MaxPooling1D, LSTM
from keras import utils
from keras.callbacks import ReduceLROnPlateau, EarlyStopping
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.losses import SparseCategoricalCrossentropy , CategoricalCrossentropy
from tensorflow.keras.utils import to_categorical
from sklearn.metrics import accuracy_score,confusion_matrix, classification_report
import warnings
warnings.filterwarnings("ignore")

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data] Downloading package omw-1.4 to /root/nltk_data...


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
df = pd.read_csv("Twitter_Data.csv")
df.head()

,clean_text,category
0,when modi promised “minimum government maximum...,-1.0
1,talk all the nonsense and continue all the dra...,0.0
2,what did just say vote for modi welcome bjp t...,1.0
3,asking his supporters prefix chowkidar their n...,1.0
4,answer who among these the most powerful world...,1.0


In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 162980 entries, 0 to 162979
Data columns (total 2 columns):
 #   Column      Non-Null Count   Dtype  
---  ------      --------------   -----  
 0   clean_text  162976 non-null  object 
 1   category    162973 non-null  float64
dtypes: float64(1), object(1)
memory usage: 2.5+ MB


In [ ]:
df.category.unique()

array([-1.,  0.,  1., nan])

In [ ]:
df.isna().sum()

clean_text    4
category      7
dtype: int64

In [ ]:
df= df.dropna()

In [ ]:
df.duplicated().sum()


0

In [ ]:
df['category'].value_counts()

 1.0    72249
 0.0    55211
-1.0    35509
Name: category, dtype: int64

In [ ]:
dist = df['category'].value_counts()
import plotly.graph_objects as go
def ditribution_plot(x,y,name):
    fig = go.Figure([
        go.Bar(x=x, y=y)
    ])

    fig.update_layout(title_text=name)
    fig.show()
ditribution_plot(x= dist.index, y= dist.values, name= 'Class Distribution train')

In [ ]:
'''def clean_text(df, field):
    df[field] = df[field].str.replace(r"@"," at ")
    df[field] = df[field].str.replace("#[^a-zA-Z0-9_]+"," ")
    df[field] = df[field].str.replace(r"[^a-zA-Z(),\"'\n_]"," ")
    df[field] = df[field].str.replace(r"http\S+","")
    df[field] = df[field].str.lower()
    return df

clean_text(df,'tweet')'''

'def clean_text(df, field):\n    df[field] = df[field].str.replace(r"@"," at ")\n    df[field] = df[field].str.replace("#[^a-zA-Z0-9_]+"," ")\n    df[field] = df[field].str.replace(r"[^a-zA-Z(),"\'\n_]"," ")\n    df[field] = df[field].str.replace(r"http\\S+","")\n    df[field] = df[field].str.lower()\n    return df\n\nclean_text(df,\'tweet\')'

In [ ]:
'''stem = PorterStemmer()
df['clean_text'] = [stem.stem(word) for word in df['clean_text'] if not word in stopwords.words('english')]'''

"stem = PorterStemmer()\ndf['clean_text'] = [stem.stem(word) for word in df['clean_text'] if not word in stopwords.words('english')]"

In [ ]:
tokenizer = Tokenizer()
tokenizer.fit_on_texts(df['clean_text'])
df['clean_text'] = tokenizer.texts_to_sequences(df['clean_text'])
df['clean_text']

0         [42, 1, 307, 42492, 66, 1726, 42493, 1119, 40,...
1         [286, 16, 2, 1104, 3, 718, 16, 2, 809, 7, 38, ...
2         [24, 53, 41, 99, 38, 4, 1, 1003, 21, 508, 5, 5...
3         [383, 19, 496, 4730, 113, 43, 1221, 1, 53, 126...
4         [390, 22, 813, 79, 2, 173, 926, 127, 136, 124,...
                                ...                        
162975    [32, 79, 5172, 474, 570, 2429, 1, 6, 2715, 17,...
162976    [305, 363, 529, 13161, 10149, 24, 36, 1, 1195,...
162977        [53, 5, 1718, 234, 2583, 7526, 118, 191, 397]
162978    [63, 183, 558, 360, 192, 9, 1, 924, 558, 15, 6...
162979    [14, 5, 260, 686, 36, 26, 35329, 118, 4882, 10...
Name: clean_text, Length: 162969, dtype: object

In [ ]:
df['category'].mask(df['category'] == 'negative',-1,  inplace=True)
df['category'].mask(df['category'] == 'normal',0,  inplace=True)
df['category'].mask(df['category'] == 'positive',1,  inplace=True)
df['category']

0        -1.0
1         0.0
2         1.0
3         1.0
4         1.0
         ... 
162975   -1.0
162976   -1.0
162977    0.0
162978    0.0
162979    1.0
Name: category, Length: 162969, dtype: float64

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(df['clean_text'],df['category'], test_size=0.2, random_state=40)

In [ ]:
print('X_train:',len(X_train))
print('y_train:',len(y_train))
print('X_test:',len(X_test))
print('y_test:',len(y_test))

X_train: 130375
y_train: 130375
X_test: 32594
y_test: 32594


In [ ]:
X_train = pad_sequences( X_train, maxlen=100 ,dtype='float32')
X_test = pad_sequences( X_test, maxlen=100 ,dtype='float32')

In [ ]:

X_test.shape

(32594, 100)

In [ ]:
len(tokenizer.index_word)

113678

In [ ]:
model_1 = Sequential()
model_1.add(Embedding(len(tokenizer.index_word)+1, input_length= 100 ,output_dim =50))
model_1.add(Bidirectional(LSTM(100)))
model_1.add(Flatten())
model_1.add(Dense(250, activation='relu'))
model_1.add(Dropout(0.2))
model_1.add(Dense(1, activation='sigmoid'))

model_1.compile("adam", loss= 'binary_crossentropy' ,metrics=["accuracy"])
model_1.summary()

Model: "sequential_10"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_10 (Embedding)    (None, 100, 50)           5683950   
                                                                 
 bidirectional_6 (Bidirectio  (None, 200)              120800    
 nal)                                                            
                                                                 
 flatten_6 (Flatten)         (None, 200)               0         
                                                                 
 dense_20 (Dense)            (None, 250)               50250     
                                                                 
 dropout_6 (Dropout)         (None, 250)               0         
                                                                 
 dense_21 (Dense)            (None, 1)                 251       
                                                     

In [ ]:
from keras.callbacks import EarlyStopping
early_stop = EarlyStopping(monitor="val_loss",patience=5,verbose=True)

In [ ]:
X_train = np.asarray(X_train).astype(np.float32)
X_test = np.asarray(X_test).astype(np.float32)

y_train = np.asarray(y_train).astype('float32').reshape((-1,1))
y_test = np.asarray(y_test).astype('float32').reshape((-1,1))

In [ ]:
X_train.shape

(130375, 100)

In [ ]:
history_1 = model_1.fit(X_train , y_train ,batch_size=64, epochs=7, validation_data=(X_test , y_test),callbacks=[early_stop])

Epoch 1/7
2038/2038 [==============================] - 392s 191ms/step - loss: -179.5327 - accuracy: 0.4431 - val_loss: -1824.0470 - val_accuracy: 0.6365
Epoch 2/7
2038/2038 [==============================] - 385s 189ms/step - loss: -22267.7637 - accuracy: 0.6039 - val_loss: -45273.4922 - val_accuracy: 0.6287
Epoch 3/7
2038/2038 [==============================] - 385s 189ms/step - loss: -94241.5625 - accuracy: 0.6141 - val_loss: -140043.9688 - val_accuracy: 0.6427
Epoch 4/7
2038/2038 [==============================] - 386s 189ms/step - loss: -214345.7500 - accuracy: 0.6403 - val_loss: -274999.2812 - val_accuracy: 0.6560
Epoch 5/7
2038/2038 [==============================] - 387s 190ms/step - loss: -392142.1562 - accuracy: 0.6534 - val_loss: -435340.3750 - val_accuracy: 0.6718
Epoch 6/7
2038/2038 [==============================] - 386s 190ms/step - loss: -595781.1875 - accuracy: 0.6464 - val_loss: -641136.2500 - val_accuracy: 0.6293
Epoch 7/7
2038/2038 [==============================] -

In [ ]:
y_train = to_categorical(y_train, 3)
y_test = to_categorical(y_test, 3)

In [ ]:
model = Sequential()
model.add(Embedding(len(tokenizer.index_word)+1, input_length= 100 ,output_dim =100))
model.add(LSTM(100))
model.add(Dense(128, activation='relu'))
model.add(Dense(3, activation='sigmoid'))

model.compile("adam", loss= 'categorical_crossentropy' ,metrics=["accuracy"])
model.summary()

Model: "sequential_11"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_11 (Embedding)    (None, 100, 100)          11367900  
                                                                 
 lstm_11 (LSTM)              (None, 100)               80400     
                                                                 
 dense_22 (Dense)            (None, 128)               12928     
                                                                 
 dense_23 (Dense)            (None, 3)                 387       
                                                                 
Total params: 11,461,615
Trainable params: 11,461,615
Non-trainable params: 0
_________________________________________________________________


In [ ]:
y_train.shape

(130375, 3)

In [ ]:
history = model.fit(X_train , y_train ,batch_size=256, epochs=4,
                    validation_data=(X_test , y_test),callbacks=[early_stop])

Epoch 1/4
510/510 [==============================] - 187s 363ms/step - loss: 0.3690 - accuracy: 0.8548 - val_loss: 0.1648 - val_accuracy: 0.9525
Epoch 2/4
510/510 [==============================] - 184s 361ms/step - loss: 0.1119 - accuracy: 0.9675 - val_loss: 0.1423 - val_accuracy: 0.9581
Epoch 3/4
510/510 [==============================] - 183s 359ms/step - loss: 0.0680 - accuracy: 0.9787 - val_loss: 0.1352 - val_accuracy: 0.9597
Epoch 4/4
510/510 [==============================] - 183s 359ms/step - loss: 0.0443 - accuracy: 0.9856 - val_loss: 0.1365 - val_accuracy: 0.9651


In [ ]:
results = model.evaluate(X_test, y_test, batch_size=64)

510/510 [==============================] - 19s 38ms/step - loss: 0.1365 - accuracy: 0.9651
